In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout,Conv2D,MaxPooling2D, BatchNormalization, LSTM
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_preprocessing.image.dataframe_iterator import DataFrameIterator
import matplotlib.pyplot as plt

In [ ]:

src = '/kaggle/input'

df1 = pd.read_csv("/kaggle/input/csv-creator-1/train.csv")
df1['image'] = 'csv-creator-1/' + df1['StudyInstanceUID'] + '-' + df1['SeriesInstanceUID'] + '-' + df1['SOPInstanceUID'] + '.png'

df2 = pd.read_csv("/kaggle/input/csv-creator-2/train.csv")
df2['image'] = 'csv-creator-2/' + df2['StudyInstanceUID'] + '-' + df2['SeriesInstanceUID'] + '-' + df2['SOPInstanceUID'] + '.png'

df3 = pd.read_csv("/kaggle/input/csv-creator-3/train.csv")
df3['image'] = 'csv-creator-3/' + df3['StudyInstanceUID'] + '-' + df3['SeriesInstanceUID'] + '-' + df3['SOPInstanceUID'] + '.png'

df4 = pd.read_csv("/kaggle/input/csv-creator-4/train.csv")
df4['image'] = 'csv-creator-4/' + df4['StudyInstanceUID'] + '-' + df4['SeriesInstanceUID'] + '-' + df4['SOPInstanceUID'] + '.png'

df5 = pd.read_csv("/kaggle/input/csv-creator-5/train.csv")
df5['image'] = 'csv-creator-5/' + df5['StudyInstanceUID'] + '-' + df5['SeriesInstanceUID'] + '-' + df5['SOPInstanceUID'] + '.png'

df6 = pd.read_csv("/kaggle/input/csv-creator-6/train.csv")
df6['image'] = 'csv-creator-6/' + df6['StudyInstanceUID'] + '-' + df6['SeriesInstanceUID'] + '-' + df6['SOPInstanceUID'] + '.png'

df7 = pd.read_csv("/kaggle/input/csv-creator-7/train.csv")
df7['image'] = 'csv-creator-7/' + df7['StudyInstanceUID'] + '-' + df7['SeriesInstanceUID'] + '-' + df7['SOPInstanceUID'] + '.png'

df = pd.concat([df1,df2,df3,df4,df5,df6,df7], ignore_index=True)#df = pd.read_csv("/kaggle/input/newcsv/new_train.csv")

#only rows needed are those with PE
#files with pe equal => 96540

x = 'image' #independent variables for DCMDataFrameIterator
y = ['pe_present_on_image'] ##dependent variables for DCMDataFrameIterator
class_mode = 'raw'

# split for testing
train_df, test_df = train_test_split(df, test_size=0.1)

# augmentation parameters
# you can use preprocessing_function instead of rescale in all generators
# if you are using a pretrained network
train_augmentation_parameters = dict(
    rescale=1.0/255.0,
    rotation_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range = [0.8, 1.2],
    validation_split = 0.2
)

valid_augmentation_parameters = dict(
    rescale=1.0/255.0,
    validation_split = 0.2
)

test_augmentation_parameters = dict(
    rescale=1.0/255.0
)

#<<==============================================================================================================>>
# training parameters
BATCH_SIZE = 32
CLASS_MODE = 'raw'
COLOR_MODE = 'rgb'
TARGET_SIZE = (512, 512)
EPOCHS = 10
SEED = 1337

train_consts = {
    'seed': SEED,
    'batch_size': BATCH_SIZE,
    'class_mode': CLASS_MODE,
    'color_mode': COLOR_MODE,
    'target_size': TARGET_SIZE,  
    'subset': 'training',
    'validate_filenames':False
}

valid_consts = {
    'seed': SEED,
    'batch_size': BATCH_SIZE,
    'class_mode': CLASS_MODE,
    'color_mode': COLOR_MODE,
    'target_size': TARGET_SIZE, 
    'subset': 'validation',
    'validate_filenames':False
}

test_consts = {
    'batch_size': 32,  # should be 1 in testing
    'class_mode': CLASS_MODE,
    'color_mode': COLOR_MODE,
    'target_size': TARGET_SIZE,  # resize input images
    'shuffle': False,
    'validate_filenames':False
}

#<<=================================================================================================================================================>>
# Using the training phase generators 
train_augmenter = ImageDataGenerator(**train_augmentation_parameters)
valid_augmenter = ImageDataGenerator(**valid_augmentation_parameters)
test_augmenter = ImageDataGenerator(**test_augmentation_parameters)


train_generator = train_augmenter.flow_from_dataframe(dataframe = train_df, directory=src, x_col=x, y_col=y, **train_consts)



valid_generator = valid_augmenter.flow_from_dataframe(dataframe = train_df, directory=src, x_col=x, y_col=y, **valid_consts)

test_generator = test_augmenter.flow_from_dataframe(dataframe = test_df, directory=src, x_col=x, y_col=y, **test_consts)


In [ ]:
#<<=================================================================================================================================================>>
# define model architecture like how you normally do
#https://www.analyticsvidhya.com/blog/2021/12/cnn-model-covid-19/


model=Sequential()

#covolution layer
model.add(Conv2D(32,(3,3),activation='relu',input_shape=[512,512,3]))
#pooling layer
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())

#covolution layer
model.add(Conv2D(32,(3,3),activation='relu'))
#pooling layer
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())

#covolution layer
model.add(Conv2D(64,(3,3),activation='relu'))
#pooling layer
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())

#covolution layer
model.add(Conv2D(64,(3,3),activation='relu'))
#pooling layer
model.add(MaxPooling2D(2,2))
model.add(BatchNormalization())

#i/p layer
model.add(Flatten())

#o/p layer
model.add(Dense(1,activation='sigmoid'))

#compile network
opt = SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['binary_accuracy'])
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())
#<<=================================================================================================================================================>>


In [ ]:
# training
# define the checkpoint
from tensorflow.keras.callbacks import ModelCheckpoint
filepath = "best_model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

#uncomment for continous training
#model = load_model('/kaggle/input/pe-results/model.h5')

history = model.fit(
train_generator,
epochs=EPOCHS,
validation_data=valid_generator,
verbose=1, 
callbacks=callbacks_list
)

In [ ]:
#commented out cuz modelcheckpoint previous block takes care of saving model
model.save('end_model.h5')
model.save_weights('weights.h5')

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator), batch_size=1)
print('test loss: {},  test_accuracy: {}'.format(test_loss, test_accuracy))

In [ ]:
#loss graph
plt.plot(history.history['loss'],label='train loss')
plt.plot(history.history['val_loss'],label='val loss')
plt.legend()

plt.savefig('loss-graph.png')
plt.show()

# accuracies
plt.plot(history.history['binary_accuracy'], label='train acc')
plt.plot(history.history['val_binary_accuracy'], label='val acc')
plt.legend()

 
plt.savefig('acc-graph.png')
plt.show()